<h1> Mission To Mars</h1>

<h4> Web scraping various sites to obtain information on Mars. </h4>

In [1]:
import html5lib
import lxml
import pandas as pd
import pymongo 
import requests 
from bs4 import BeautifulSoup as bs
from selenium import webdriver

In [2]:
hemispheres_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"

<h3>Nasa Mars News</h3>

Had issues getting Requests to pull the info so I had to use Selenium + lxml

In [3]:
# scrape Mars News 
nasa_url = 'https://mars.nasa.gov/news/'

driver = webdriver.Firefox()
driver.get(nasa_url)
driver.implicitly_wait(5)

nasa_soup = bs(driver.page_source,'html.parser')
driver.close()

In [4]:
# find latest news title and paragraph text
news_title = nasa_soup.body.find('div', class_='content_title').text
news_paragraph = nasa_soup.body.find('div', class_='article_teaser_body').text

print(news_title)
print(news_paragraph)

The MarCO Mission Comes to an End
The pair of briefcase-sized satellites made history when they sailed past Mars in 2019.


<h3>JPL Mars Space Images - Featured Image</h3>

In [5]:
# scrape JPL
jpl_base = 'https://www.jpl.nasa.gov'
jpl_url = jpl_base+'/spaceimages/?search=&category=Mars'
response = requests.get(jpl_url)

jpl_soup = bs(response.text, 'html.parser')

In [6]:
# find featured image 
jpl_soup.find('article')['style']

"background-image: url('/spaceimages/images/wallpaper/PIA18846-1920x1200.jpg');"

In [7]:
# strip off everything that isnt the image path  
image_str = jpl_soup.find('article')['style'].split(' ')[1].strip("url").strip("('');")
image_str

'/spaceimages/images/wallpaper/PIA18846-1920x1200.jpg'

In [8]:
# concatenate base url with image path 
featured_image_url = jpl_base + image_str
featured_image_url

'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA18846-1920x1200.jpg'

<h3>Mars Weather</h3>

In [9]:
# scrape Twitter
weather_url = 'https://twitter.com/marswxreport?lang=en'
response = requests.get(weather_url)

weather_soup = bs(response.text, 'html.parser')

In [19]:
# find most recent weather report  
tweets = weather_soup.find_all('div', class_ = 'js-tweet-text-container')

for tweet in tweets:
    mars_weather = tweet.find('p').text
    break

print(mars_weather)

360 Sol 2687 selfiehttps://roundme.com/tour/550044/view?welcome …


<h3>Mars Facts</h3>

In [11]:
# scrape Mars Facts 
facts_url = 'http://space-facts.com/mars/'
response = requests.get(facts_url)

facts_soup = bs(response.text, 'html.parser')

In [12]:
# find table and read it into a pandas dataframe 
table = facts_soup.find_all('table')[0]
table

df = pd.read_html(str(table))
df

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers]

<h3>Mars Hemispheres</h3>

In [13]:
# scrape -- Requests 
hemi_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
response = requests.get(hemi_url)

hemi_soup = bs(response.text, 'html.parser')

In [14]:
hemi_base = 'https://astrogeology.usgs.gov'

hemi_imgs = []
hemi_imgs = [
    {
        "Title": hemi.text.strip(" Enhanced"),
        "URL": (hemi_base + hemi["href"]),
    }
    for hemi in hemi_soup.find_all('a', class_='itemLink product-item')
]
hemi_imgs

[{'Title': 'Cerberus Hemispher',
  'URL': 'https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced'},
 {'Title': 'Schiaparelli Hemispher',
  'URL': 'https://astrogeology.usgs.gov/search/map/Mars/Viking/schiaparelli_enhanced'},
 {'Title': 'Syrtis Major Hemispher',
  'URL': 'https://astrogeology.usgs.gov/search/map/Mars/Viking/syrtis_major_enhanced'},
 {'Title': 'Valles Marineris Hemispher',
  'URL': 'https://astrogeology.usgs.gov/search/map/Mars/Viking/valles_marineris_enhanced'}]

In [15]:
# clickable URLs
images = hemi_soup.find_all(class_='itemLink product-item')
hemis = ["Cerberus Hemisphere", "Schiaparelli Hemisphere", "Syrtis Major Hemisphere", "Valles Marineris Hemisphere"]

for h in hemis:
    print(h)
    
for image in images:
    image_url = hemi_base + image['href']
    print(image_url)      

Cerberus Hemisphere
Schiaparelli Hemisphere
Syrtis Major Hemisphere
Valles Marineris Hemisphere
https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced
https://astrogeology.usgs.gov/search/map/Mars/Viking/schiaparelli_enhanced
https://astrogeology.usgs.gov/search/map/Mars/Viking/syrtis_major_enhanced
https://astrogeology.usgs.gov/search/map/Mars/Viking/valles_marineris_enhanced


In [16]:
def get_images(hemis):
    return hemis 

get_images(hemis)

['Cerberus Hemisphere',
 'Schiaparelli Hemisphere',
 'Syrtis Major Hemisphere',
 'Valles Marineris Hemisphere']